In [69]:
import requests
import json
import pandas as pd
import tableauserverclient as TSC
from pandleau import *

r = requests.get(url = 'https://sandbox-api.brewerydb.com/v2/locations',
                params = {'key':'Your-Key-Here'})
                 
data = r.json()
data = data['data']

df = pd.DataFrame(data)
df
for i in df.country:
    df['country'] = i['name']
    
    
brewery_name = {}
brewery_desc = {}
established = {}
organic = {}
status = {}
massOwned = {}
inBusiness = {}
for b in df.brewery:
    try:
        brewery_name[b['id']] = b['name']
        brewery_desc[b['id']] = b['description']
        established[b['id']] = b['established']
        organic[b['id']] = b['isOrganic']
        status[b['id']] = b['status']
        massOwned[b['id']] = b['isMassOwned']
        inBusiness[b['id']] = b['inBusiness']
    except:
        continue
        
df['brewery_name'] = df['breweryId'].map(brewery_name)
df['brewery_desc'] = df['breweryId'].map(brewery_desc)
df['established'] = df['breweryId'].map(established)
df['organic'] = df['breweryId'].map(organic)
df['status'] = df['breweryId'].map(status)
df['massOwned'] = df['breweryId'].map(massOwned)
df['inBusiness'] = df['breweryId'].map(inBusiness)

brewery_cols = ['breweryId', 'country', 'countryIsoCode', 'createDate', 'extendedAddress',
       'id', 'inPlanning', 'isClosed', 'isPrimary', 'latitude', 'locality', 'locationType', 
       'locationTypeDisplay', 'longitude', 'name', 'openToPublic', 'phone', 'postalCode', 
       'region', 'statusDisplay', 'streetAddress', 'timezoneId', 'updateDate', 'website',
       'yearOpened', 'brewery_name', 'brewery_desc', 'established', 'organic', 'massOwned', 
       'inBusiness']

final_brewery_df = df[brewery_cols]
final_brewery_df = pandleau(final_brewery_df)
final_brewery_df.to_tableau(r'\Users\MSomma\Documents\BreweryDB\BreweryDB_Example.hyper', add_index=False)

tableau_auth = TSC.TableauAuth('Your-Tableau-Online-User-Name-Here',
                               'Your-Tableau-Online-Password-Here', site_id = 'Your-Tableau-Site-Name-Here')
server = TSC.Server('Your-Server-Address-Here',
                    use_server_version=True)
server.auth.sign_in(tableau_auth)

projectname = 'Breweries in America'
print([(project.name, project.id) for project in TSC.Pager(server.projects) if project.name.startswith(projectname)])

mydatasourceitem = TSC.DatasourceItem('fcf8d4e0-8028-41e4-96f2-50902996f669',
                                      name='Brewery DB')
item = server.datasources.publish(mydatasourceitem,
                                  r'\Users\MSomma\Documents\BreweryDB\BreweryDB.hyper', 'Overwrite')
print("{} published with id: {}".format(item.name, item.id))

server.auth.sign_out()

processing table: 42it [00:00, 1550.12it/s]


[('Breweries in America', 'fcf8d4e0-8028-41e4-96f2-50902996f669')]
Brewery DB published with id: 52e3ae17-0e5c-4a08-a074-ec3f47ec425c
